In [1]:
def data_read(filename):
    intent = []
    intent_unique = set()
    data = []
    vocab = {}
    start = True
    sentence = ""
    with open(filename, 'r') as f:
        for line in f:
            line = line.split()
            if(start):
                temp = line[8].split("+")
                intent.append(temp[0])
                intent_unique.add(temp[0])
                start = False
            else:
                if(line[4] != 'EOS'):
                    sentence += " " + line[4]
                    if line[4] not in vocab:
                        vocab[line[4]] = len(vocab)
                else:
                    start = True
                    data.append(sentence[1:])
                    sentence = ""
    return intent, intent_unique, data, vocab

def unigram_vector(data, vocab):
    unigrams = []
    for sentence in data:
        vec = [0]*len(vocab)
        sentence = sentence.split()
        for word in sentence:
            if word not in vocab:
                continue
            vec[vocab[word]] = 1
        unigrams.append(vec)
    return unigrams

def to_dict(set_data):
    dictionary = {}
    for i in set_data:
        dictionary[i] = len(dictionary)
    return dictionary

In [2]:
train_intent, intent_unique, train_data, vocab = data_read("atis.train.ctf")
intent_unique = to_dict(intent_unique)
intent_unique['other'] = len(intent_unique)
for i in range(len(train_intent)):
    train_intent[i] = intent_unique[train_intent[i]]
train_unigrams = unigram_vector(train_data, vocab)

In [3]:
test_intent, temp1, test_data, temp2 = data_read("atis.test.ctf")
for i in range(len(test_intent)):
    if test_intent[i] not in intent_unique:
        test_intent[i] = intent_unique['other']
    else:
        test_intent[i] = intent_unique[test_intent[i]]
test_unigrams = unigram_vector(test_data, vocab)

In [4]:
input_dimension = len(train_unigrams[0])

In [5]:
import numpy as np
train_features = np.array(train_unigrams)
train_labels = np.array(train_intent)
test_features = np.array(test_unigrams)
test_labels = np.array(test_intent)

In [ ]:
from keras.utils import np_utils
train_labels = np_utils.to_categorical(train_labels,len(intent_unique))
test_labels = np_utils.to_categorical(test_labels,len(intent_unique))

In [ ]:
from keras.models import Sequential
from keras.layers import Activation, Dense
from keras.optimizers import SGD, Adam
from sklearn.metrics import precision_recall_fscore_support
def model(optimizer, loss, activation, epochs, x_train, y_train, x_test, y_test, test_intent):
    model = Sequential()
    model.add(Dense(100, activation=activation, input_dim=x_train.shape[1]))
    model.add(Dense(len(y_train[0])))
    model.add(Activation("softmax"))
    if optimizer == "adam":
        opt = Adam()
    if optimizer == "sgd":
        opt  = SGD()
    model.compile(loss=loss, optimizer=opt, metrics=["accuracy"])
    print("Training")
    model.fit(x_train, y_train, epochs=epochs, batch_size=64, verbose=1)
    print("Testing")
    loss, accuracy = model.evaluate(x_test, y_test, batch_size=32, verbose=1)
    print("[INFO] loss={:.4f}, accuracy: {:.4f}%".format(loss, accuracy * 100))
#     print(test_intent)
    tuple_scores = precision_recall_fscore_support(test_intent, np.argmax(model.predict(x_test), axis = 1), average='weighted')
    print("precision: "+ str(tuple_scores[0])+" recall: "+str(tuple_scores[1])+ " f-score: "+str(tuple_scores[2]))
    return model

In [ ]:
classifier = model("adam", "categorical_crossentropy", "tanh", 10, train_features, train_labels, test_features, test_labels, test_intent)

In [ ]:
test_sentence = "my flight day"
list_of_sen = [test_sentence]
test_uni = unigram_vector(list_of_sen, vocab)
print(intent_unique)
np.argmax(classifier.predict(np.array(test_uni)))
